In this notebook we give a proof of concept of unitary compiling using TFQ. 

In [16]:
%load_ext autoreload
%autoreload 2

import sympy 
import numpy as np 
import pandas as pd 
import tensorflow as tf
from utilities.circuit_database import CirqTranslater
from utilities.templates import *
from utilities.variational import Minimizer
import matplotlib.pyplot as plt 
import tensorflow_quantum as tfq
import cirq
from utilities.compiling import *
from utilities.misc import *
from utilities.simplifier import Simplifier
from utilities.discrimination import *
from utilities.idinserter import IdInserter
from utilities.evaluator import Evaluator
from utilities.gate_killer import GateKiller
from ast import literal_eval


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
np.random.seed(0)
translator = CirqTranslater(4, untouchable_blocks = [])

translator.env_qubits = [2]

simplifier = Simplifier(translator)
etas = [0.01, 1.]
minimizer = Minimizer(translator, mode="discrimination", params=etas)

q = 0
other_qs = list(range(translator.n_qubits))
other_qs.remove(q)
indds = [translator.cnots_index[str([q,k])] for k in other_qs]
cdb = pd.DataFrame([gate_template(i) for i in indds])

circuit_db = concatenate_dbs([z_layer_db(translator), cdb])

for q in range(1, translator.n_qubits):
    other_qs = list(range(translator.n_qubits))
    other_qs.remove(q)
    indds = [translator.cnots_index[str([q,k])] for k in other_qs]
    cdb = pd.DataFrame([gate_template(i) for i in indds])
    circuit_db = concatenate_dbs([circuit_db ,cdb])

c, circuit_db = translator.give_circuit(circuit_db)

db1 = concatenate_dbs([x_layer_db(translator)]*3)
#db2 = concatenate_dbs([z_layer_db(translator)]*3)
#db3 = concatenate_dbs([x_layer_db(translator)]*3)
#db4 = concatenate_dbs([z_layer_db(translator)]*3)

circuit_db = concatenate_dbs([circuit_db, db1])
c, circuit_db = translator.give_circuit(circuit_db)

cdb = []
for ind, qubits in list(translator.indexed_cnots.items()):
    cdb.append(gate_template(int(ind), block_id=0))
    con, tar = qubits
    cdb.append(gate_template(int(con + translator.number_of_cnots), param_value = np.random.random()))
    cdb.append(gate_template(int(tar + translator.number_of_cnots + translator.n_qubits), param_value = np.random.random()))
c6_db = pd.DataFrame(cdb)
circuit_db = concatenate_dbs([circuit_db, c6_db])

c, circuit_db = translator.give_circuit(circuit_db)


dd1  =circuit_db.copy()
dd1["block_id"] = 1
dd1["symbol"] = None
circuit_db = concatenate_dbs([circuit_db, dd1])
c, circuit_db = translator.give_circuit(circuit_db)

simplifier = Simplifier(translator, apply_relatives_to_first = True)
simplifier.apply_relatives_to_first = True
simplifier.relative_rules = [simplifier.rule_1]
simplified_db, ns = simplifier.reduce_circuit(circuit_db)
ss, simplified_db = translator.give_circuit(simplified_db)

simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7fb34a696fd0>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7fb34a696fd0>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7fb34a696fd0>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7fb34a696fd0>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7fb34a696fd0>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7fb34a696fd0>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7fb34a696fd0>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7fb34a696fd0>>
simplified using  <bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 

simplified using  <bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7fb34a696fd0>>


In [18]:
mdif = max_diff(translator , circuit_db, simplified_db)
print("\n")
print(mdif)
print(c)
print()
print(ss)



4.308824046522781e-16
                                          ┌──┐                                                                                                                                                     ┌──┐                                                                                                                                           ┌──┐                                                                                                                                                       ┌──┐
(0, 0): ───Rz(th_0)───@───@───@───X─────────X────────────X───Rx(th_4)───Rx(th_8)───Rx(th_12)───────────────@───────────Rz(th_16)───@───Rz(th_18)───@───Rz(th_20)───X───Rx(th_23)─────────────────────X────Rx(th_29)───────────────────────────────────X───Rx(th_35)───Rz(th_40)───────────────────────────@───────────────────────@───@───X─────────X────────────X───Rx(th_44)───Rx(th_48)───Rx(th_52)───────────────@───────────Rz(th_56)───@───Rz(th_58)───@───Rz(th_60)───X───Rx(th_63)────

In [19]:
len(simplified_db), len(circuit_db)

(109, 128)

In [20]:
np.random.seed(0)
translator = CirqTranslater(4, untouchable_blocks = [0])

translator.env_qubits = [2]

simplifier = Simplifier(translator)
etas = [0.01, 1.]
minimizer = Minimizer(translator, mode="discrimination", params=etas)

q = 0
other_qs = list(range(translator.n_qubits))
other_qs.remove(q)
indds = [translator.cnots_index[str([q,k])] for k in other_qs]
cdb = pd.DataFrame([gate_template(i) for i in indds])

circuit_db = concatenate_dbs([z_layer_db(translator), cdb])

for q in range(1, translator.n_qubits):
    other_qs = list(range(translator.n_qubits))
    other_qs.remove(q)
    indds = [translator.cnots_index[str([q,k])] for k in other_qs]
    cdb = pd.DataFrame([gate_template(i) for i in indds])
    circuit_db = concatenate_dbs([circuit_db ,cdb])

c, circuit_db = translator.give_circuit(circuit_db)

db1 = concatenate_dbs([x_layer_db(translator)]*3)
#db2 = concatenate_dbs([z_layer_db(translator)]*3)
#db3 = concatenate_dbs([x_layer_db(translator)]*3)
#db4 = concatenate_dbs([z_layer_db(translator)]*3)

circuit_db = concatenate_dbs([circuit_db, db1])
c, circuit_db = translator.give_circuit(circuit_db)

cdb = []
for ind, qubits in list(translator.indexed_cnots.items()):
    cdb.append(gate_template(int(ind), block_id=0))
    con, tar = qubits
    cdb.append(gate_template(int(con + translator.number_of_cnots), param_value = np.random.random()))
    cdb.append(gate_template(int(tar + translator.number_of_cnots + translator.n_qubits), param_value = np.random.random()))
c6_db = pd.DataFrame(cdb)
circuit_db = concatenate_dbs([circuit_db, c6_db])

c, circuit_db = translator.give_circuit(circuit_db)


dd1  =circuit_db.copy()
dd1["block_id"] = 2
dd1["symbol"] = None
circuit_db = concatenate_dbs([circuit_db, dd1])
c, circuit_db = translator.give_circuit(circuit_db)

simplifier = Simplifier(translator, apply_relatives_to_first = True)
simplifier.apply_relatives_to_first = True
simplifier.relative_rules = [simplifier.rule_1]
simplified_db, ns = simplifier.reduce_circuit(circuit_db)
ss, simplified_db = translator.give_circuit(simplified_db)

simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7fb27a92ed68>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7fb27a92ed68>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7fb27a92ed68>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7fb27a92ed68>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7fb27a92ed68>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7fb27a92ed68>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7fb27a92ed68>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7fb27a92ed68>>
simplified using  <bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 

In [21]:
np.random.seed(0)
translator = CirqTranslater(4, untouchable_blocks = [0])

translator.env_qubits = [2]

simplifier = Simplifier(translator)
etas = [0.01, 1.]
minimizer = Minimizer(translator, mode="discrimination", params=etas)


ddb =concatenate_dbs([x_layer_db(translator)]*2 +[z_layer_db(translator)]*2 + [x_layer_db(translator)]*1 + [x_layer_db(translator)]*2 +  [z_layer_db(translator)]*2 )

cnots = list(np.squeeze(np.array([[k,k] for k in range(translator.number_of_cnots)]).reshape([-1,1])))
ddbc = pd.DataFrame([gate_template(k) for k in cnots])

cdb = concatenate_dbs([ddbc, ddb])
circuit, circuit_db = translator.give_circuit(cdb)


dd1  =circuit_db.copy()
dd1["block_id"] = 1
dd1["symbol"] = None
circuit_db = concatenate_dbs([circuit_db, dd1])
c, circuit_db = translator.give_circuit(circuit_db)

simplifier = Simplifier(translator, apply_relatives_to_first = True)
simplifier.apply_relatives_to_first = True
simplifier.relative_rules = [simplifier.rule_1]
simplified_db, ns = simplifier.reduce_circuit(circuit_db)
ss, simplified_db = translator.give_circuit(simplified_db)

simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7fb27a8cb7b8>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7fb27a8cb7b8>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7fb27a8cb7b8>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7fb27a8cb7b8>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7fb27a8cb7b8>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7fb27a8cb7b8>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7fb27a8cb7b8>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7fb27a8cb7b8>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 

In [22]:
mdif = max_diff(translator , circuit_db, simplified_db)
print("\n")
print(mdif)
print(c)
print()
print(ss)



0.00030185333546601135
                                                   ┌──┐   ┌──┐                                                                                                                                                                                                                    ┌──┐   ┌──┐
(0, 0): ───@───@───@───@───@───@───X───X─────────────X──────X────────────────────X───X───Rx(th_0)───Rx(th_4)───Rz(th_8)───Rz(th_12)───Rx(th_16)───Rx(th_20)───Rx(th_24)───Rz(th_28)───Rz(th_32)───────────────────────────@───────────@───────────@───@───@───@───X───X─────────────X──────X────────────────────X───X───Rx(th_36)───Rx(th_40)───Rz(th_44)───Rz(th_48)───Rx(th_52)───Rx(th_56)───Rx(th_60)───Rz(th_64)───Rz(th_68)───────────────────────────────────────────────────
           │   │   │   │   │   │   │   │             │      │                    │   │                                                                                                                                    │ 

In [8]:
len(circuit_db), len(simplified_db)

(120, 72)

In [9]:
np.random.seed(0)
translator = CirqTranslater(4, untouchable_blocks = [0,1])

translator.env_qubits = [2]

simplifier = Simplifier(translator)
etas = [0.01, 1.]
minimizer = Minimizer(translator, mode="discrimination", params=etas)


ddb =concatenate_dbs([x_layer_db(translator)]*2 +[z_layer_db(translator)]*2 + [x_layer_db(translator)]*1 + [x_layer_db(translator)]*2 +  [z_layer_db(translator)]*2 )

cnots = list(np.squeeze(np.array([[k,k] for k in range(translator.number_of_cnots)]).reshape([-1,1])))
ddbc = pd.DataFrame([gate_template(k) for k in cnots])

cdb = concatenate_dbs([ddbc, ddb])
circuit, circuit_db = translator.give_circuit(cdb)


dd1  =circuit_db.copy()
dd1["block_id"] = 1
dd1["symbol"] = None
circuit_db = concatenate_dbs([circuit_db, dd1])
c, circuit_db = translator.give_circuit(circuit_db)


dd1  =circuit_db.copy()
dd1["block_id"] = 2
dd1["symbol"] = None
circuit_db = concatenate_dbs([circuit_db, dd1])
c, circuit_db = translator.give_circuit(circuit_db)

simplifier = Simplifier(translator, apply_relatives_to_first = True)
simplifier.apply_relatives_to_first = True
simplifier.relative_rules = [simplifier.rule_1]
simplified_db, ns = simplifier.reduce_circuit(circuit_db)
ss, simplified_db = translator.give_circuit(simplified_db)

simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7fb2bc039a20>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7fb2bc039a20>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7fb2bc039a20>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7fb2bc039a20>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7fb2bc039a20>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7fb2bc039a20>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7fb2bc039a20>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7fb2bc039a20>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 

AttributeError: ojo <bound method Simplifier.rule_5 of <utilities.simplifier.Simplifier object at 0x7fb2bc039a20>>

In [10]:
mdif = max_diff(translator , circuit_db, simplified_db)
print("\n")
print(mdif)
print(c)
print()
print(ss)



1.1215123277163865
                                                   ┌──┐   ┌──┐                                                                                                                                                                                                                    ┌──┐   ┌──┐                                                                                                                                                                                                                       ┌──┐   ┌──┐                                                                                                                                                                                                                             ┌──┐   ┌──┐
(0, 0): ───@───@───@───@───@───@───X───X─────────────X──────X────────────────────X───X───Rx(th_0)───Rx(th_4)───Rz(th_8)───Rz(th_12)───Rx(th_16)───Rx(th_20)───Rx(th_24)───Rz(th_28)───Rz(th_32)───────────────────────────@───────────@────

In [11]:
import pandas as pd
cc = pd.read_csv("testing/data/dcl")

In [12]:
ppp = cc.drop("Unnamed: 0", axis=1)

In [14]:
ss,ddb = translator.give_circuit(ppp)

warning, repeated symbol while constructing the circuit, see circuut_
  symbol_name th_72
 symbols ['th_72', 'th_73', 'th_74', 'th_75', 'th_76', 'th_77', 'th_78', 'th_79', 'th_80', 'th_81', 'th_82', 'th_83', 'th_84', 'th_85', 'th_86', 'th_87', 'th_88', 'th_89', 'th_90', 'th_91', 'th_92', 'th_93', 'th_94', 'th_95', 'th_96', 'th_97', 'th_98', 'th_99', 'th_100', 'th_101', 'th_102', 'th_103']
circuit_db {0: {'ind': 16, 'symbol': 'th_72', 'param_value': 0.0, 'trainable': True, 'block_id': 2}, 1: {'ind': 17, 'symbol': 'th_73', 'param_value': 0.0, 'trainable': True, 'block_id': 2}, 2: {'ind': 18, 'symbol': 'th_74', 'param_value': 0.0, 'trainable': True, 'block_id': 2}, 3: {'ind': 19, 'symbol': 'th_75', 'param_value': 0.0, 'trainable': True, 'block_id': 2}, 4: {'ind': 12, 'symbol': 'th_76', 'param_value': 3.448296944257913, 'trainable': True, 'block_id': 2}, 5: {'ind': 13, 'symbol': 'th_77', 'param_value': 4.493667318642263, 'trainable': True, 'block_id': 2}, 6: {'ind': 14, 'symbol': 'th_78', 

In [15]:
ss

(0, 0): ───Rx(th_72)───Rz(th_76)───Rx(th_80)───Rz(th_84)───Rx(th_88)───Rz(th_92)───Rx(th_96)───Rz(th_100)───Rz(th_72)───Rx(th_76)───Rz(th_80)───

(0, 1): ───Rx(th_73)───Rz(th_77)───Rx(th_81)───Rz(th_85)───Rx(th_89)───Rz(th_93)───Rx(th_97)───Rz(th_101)───────────────────────────────────────

(0, 2): ───Rx(th_74)───Rz(th_78)───Rx(th_82)───Rz(th_86)───Rx(th_90)───Rz(th_94)───Rx(th_98)───Rz(th_102)───────────────────────────────────────

(0, 3): ───Rx(th_75)───Rz(th_79)───Rx(th_83)───Rz(th_87)───Rx(th_91)───Rz(th_95)───Rx(th_99)───Rz(th_103)───────────────────────────────────────

ss

In [21]:
np.random.seed(0)
translator = CirqTranslater(4, untouchable_blocks = [])

translator.env_qubits = [2]

simplifier = Simplifier(translator)
etas = [0.01, 1.]
minimizer = Minimizer(translator, mode="discrimination", params=etas)


ddb =concatenate_dbs([x_layer_db(translator)]*2 +[z_layer_db(translator)]*2 + [x_layer_db(translator)]*1 + [x_layer_db(translator)]*2 +  [z_layer_db(translator)]*2 )

cnots = list(np.squeeze(np.array([[k,k] for k in range(translator.number_of_cnots)]).reshape([-1,1])))
ddbc = pd.DataFrame([gate_template(k) for k in cnots])

cdb = concatenate_dbs([ddbc, ddb])
circuit, circuit_db = translator.give_circuit(cdb)

simplifier = Simplifier(translator, apply_relatives_to_first = True)
simplifier.apply_relatives_to_first = True
simplifier.relative_rules = [simplifier.rule_1]
simplified_db, ns = simplifier.reduce_circuit(circuit_db)
ss, simplified_db = translator.give_circuit(simplified_db)

simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7f5aa3a80080>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7f5aa3a80080>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7f5aa3a80080>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7f5aa3a80080>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7f5aa3a80080>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7f5aa3a80080>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7f5aa3a80080>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7f5aa3a80080>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 

In [24]:
mdif = max_diff(translator , circuit_db, simplified_db)
print("\n")
print(mdif)
print(c)
print()
print(ss)



0.0003018533354660049
                                                   ┌──┐   ┌──┐                                                                                                                                                                                                                    ┌──┐   ┌──┐                                                                                                                                                                                                                       ┌──┐   ┌──┐                                                                                                                                                                                                                             ┌──┐   ┌──┐
(0, 0): ───@───@───@───@───@───@───X───X─────────────X──────X────────────────────X───X───Rx(th_0)───Rx(th_4)───Rz(th_8)───Rz(th_12)───Rx(th_16)───Rx(th_20)───Rx(th_24)───Rz(th_28)───Rz(th_32)───────────────────────────@───────────@─

In [38]:
np.random.seed(0)
translator = CirqTranslater(4, untouchable_blocks = [0,1])

translator.env_qubits = [2]

simplifier = Simplifier(translator)
etas = [0.01, 1.]
minimizer = Minimizer(translator, mode="discrimination", params=etas)

ddb =concatenate_dbs([x_layer_db(translator)]*2 +[z_layer_db(translator)]*2 + [x_layer_db(translator)]*1 + [x_layer_db(translator)]*2 +  [z_layer_db(translator)]*2 )

cnots = list(np.squeeze(np.array([[k,k] for k in range(translator.number_of_cnots)]).reshape([-1,1])))
ddbc = pd.DataFrame([gate_template(k) for k in cnots])

cdb = concatenate_dbs([ddbc, ddb])
circuit, circuit_db = translator.give_circuit(cdb)

dd1  =circuit_db.copy()
dd1["block_id"] = 1
dd1["symbol"] = None
circuit_db = concatenate_dbs([circuit_db, dd1])
c, circuit_db = translator.give_circuit(circuit_db)


dd1  =circuit_db.copy()
dd1["block_id"] = 2
dd1["symbol"] = None
circuit_db = concatenate_dbs([circuit_db, dd1])
c, circuit_db = translator.give_circuit(circuit_db)



simplifier = Simplifier(translator, apply_relatives_to_first = True)
simplifier.apply_relatives_to_first = True
simplifier.relative_rules = [simplifier.rule_1]
simplified_db, ns = simplifier.reduce_circuit(circuit_db)
ss, simplified_db = translator.give_circuit(simplified_db)

simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7f5ab3633470>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7f5ab3633470>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7f5ab3633470>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7f5ab3633470>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7f5ab3633470>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7f5ab3633470>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7f5ab3633470>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 0x7f5ab3633470>>
simplified using  <bound method Simplifier.rule_2 of <utilities.simplifier.Simplifier object at 

AttributeError: ojo <bound method Simplifier.rule_5 of <utilities.simplifier.Simplifier object at 0x7f5ab3633470>>

In [37]:
mdif = max_diff(translator , circuit_db, simplified_db)
print("\n")
print(mdif)
print(c)
print()
print(ss)



0.000301853335466016
                                                   ┌──┐   ┌──┐                                                                                                                                                                                                                    ┌──┐   ┌──┐                                                                                                                                                                                                                       ┌──┐   ┌──┐                                                                                                                                                                                                                             ┌──┐   ┌──┐
(0, 0): ───@───@───@───@───@───@───X───X─────────────X──────X────────────────────X───X───Rx(th_0)───Rx(th_4)───Rz(th_8)───Rz(th_12)───Rx(th_16)───Rx(th_20)───Rx(th_24)───Rz(th_28)───Rz(th_32)───────────────────────────@───────────@──

In [33]:
check_symbols_ordered(simplified_db)

True